<a href="https://colab.research.google.com/github/lorenzozanisi/AI2SWINTERDATATHON_PROBLEM/blob/main/TFP_VAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import tensorflow.keras.layers as tfkl
import pandas as pd
import matplotlib.pylab as plt
import numpy as np
from tensorflow.keras import Model
import tensorflow_probability as tfp
from tensorflow_probability import distributions as tfd
from tensorflow_probability import layers as tfpl
import tensorflow as tf
from scipy.special import factorial
from google.colab import files
import shutil

In [2]:
tf.config.run_functions_eagerly(True)
%load_ext tensorboard

In [3]:
data = files.upload()

Saving coad_major.txt to coad_major (1).txt


In [56]:
data = pd.read_csv('coad_major.txt', sep=',', header=None)

data = data.T #rows= sequence, y= position in the genome

train_size = int(len(data)*0.75)
train = np.array(data.iloc[:train_size]).astype(np.float32)
test = np.array(data.iloc[train_size:]).astype(np.float32)

In [59]:
np.array([train,train]).shape   ####WORK ON THIS!!

(2, 285, 10000)

In [5]:
input_shape = data.shape[1]
batch_size = 16
train = tf.data.Dataset.from_tensor_slices(train).shuffle(train_size).batch(batch_size)
test = tf.data.Dataset.from_tensor_slices(test).shuffle(train_size).batch(batch_size)

In [6]:
train.take(1)

<TakeDataset shapes: (None, 10000), types: tf.float32>

In [7]:
# from TFP API, https://www.tensorflow.org/probability/examples/Probabilistic_Layers_VAE

In [8]:
latent_dim = 16
prior = tfd.Independent(tfd.Normal(loc=tf.zeros(latent_dim),scale=1),reinterpreted_batch_ndims=1)


In [37]:
encoder = tf.keras.Sequential(
            [tfkl.InputLayer(input_shape=input_shape),
             tfkl.Dense(5000, activation='relu',name='encoder_dense1'),
             tfkl.Dense(1000, activation='relu', name='encoder_dense2'),
             tfkl.Dense(500, activation='relu',name='encoder_dense3'),
             tfkl.Dense(tfpl.MultivariateNormalTriL.params_size(latent_dim), activation='relu',name='encoder_dense4'), #the number of parameters that are needed to describe the multivariate
             tfpl.MultivariateNormalTriL(latent_dim, activity_regularizer=tfpl.KLDivergenceRegularizer(prior))
            ])



decoder = tf.keras.Sequential(
            [tfkl.InputLayer(input_shape=[latent_dim]),
             tfkl.Dense(500,activation='relu'),
             tfkl.Dense(1000, activation='relu'),
             tfkl.Dense(5000, activation='relu'),
             tfkl.Dense(tfpl.IndependentPoisson.params_size(input_shape), activation='relu'),
             tfpl.IndependentPoisson(input_shape)        
            ])

In [24]:
model = tf.keras.Model(inputs=encoder.inputs, outputs=decoder(encoder.outputs[0]))

In [15]:
negloglik = lambda x,rv : -rv.log_prob(x)

In [25]:

model.compile(optimizer=tf.optimizers.Adam(learning_rate=1e-3),
            loss=negloglik)


In [28]:
model.fit(train, epochs=1)

18/18 [==============================] - 10s 535ms/step - loss: 3.2267


In [29]:
model.variables

[<tf.Variable 'encoder_dense1/kernel:0' shape=(10000, 5000) dtype=float32, numpy=
 array([[ 0.00957055, -0.01487634, -0.01079694, ...,  0.00116258,
         -0.01720769, -0.01758911],
        [ 0.00143564, -0.00455323,  0.01307984, ...,  0.0103692 ,
          0.00580201,  0.00595688],
        [-0.00117194,  0.00762449, -0.00239741, ..., -0.0004887 ,
          0.00967936, -0.0019867 ],
        ...,
        [ 0.01595523,  0.0077684 ,  0.01204156, ..., -0.0113468 ,
          0.01036847,  0.00789162],
        [ 0.00643632,  0.00841137, -0.01422711, ..., -0.0040859 ,
         -0.0217581 ,  0.01378467],
        [ 0.00572278,  0.0077914 , -0.02123341, ..., -0.02331766,
         -0.00293705, -0.02038405]], dtype=float32)>,
 <tf.Variable 'encoder_dense1/bias:0' shape=(5000,) dtype=float32, numpy=
 array([ 0.        ,  0.00212105, -0.00600504, ..., -0.00466226,
        -0.00600503,  0.00028827], dtype=float32)>,
 <tf.Variable 'encoder_dense2/kernel:0' shape=(5000, 1000) dtype=float32, numpy=
 ar

In [26]:

hist = model.fit(train, epochs=15, callbacks=[tf.keras.callbacks.TensorBoard(log_dir='tfpvae')] , validation_data=test)

Epoch 1/15
18/18 [==============================] - 11s 621ms/step - loss: 3.5448 - val_loss: 3.6184
Epoch 2/15
18/18 [==============================] - 11s 641ms/step - loss: 3.0018 - val_loss: 3.6869
Epoch 3/15
14/18 [======================>.......] - ETA: 2s - loss: 3.2291

KeyboardInterrupt: ignored

In [ ]:
model.layers